In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
import glob
import pickle
import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from models import CNN_LSTM, SepCNN_LSTM, ConvGRU_LSTM, RandomForestBaseline, LassoModel
import joblib

In [2]:
seed = 42

# Set seed for NumPy
np.random.seed(seed)

# Set seed for Python's built-in random module
random.seed(seed)

# Set seed for PyTorch
torch.manual_seed(seed)

# Set seed for Torch's CUDA operations if GPU is used
# if torch.cuda.is_available():
#     torch.backends.cudnn.deterministic = True
#     torch.cuda.manual_seed(seed)

In [6]:
# # Load a sample of the data
sample_data = np.load('./data/PROCESSED_III/2021_48_481.npy')  

# # Check the shape of the sample data
print("Shape of sample data:", sample_data.shape)

Shape of sample data: (38, 1, 128, 9)


In [3]:
# Define generator function
def generator(IDs, yields, batch_size, cutoff=None):
    def load_data(ID):
        try:
            data = np.load('./data/PROCESSED_III/' + ID + '.npy')
            return data
        except Exception as e:
            # print('Error loading data:', e)
            return None

    batches = 0

    while True:
        batch_features = np.zeros((batch_size, 38, 1, 128, 9)) if cutoff is None else np.zeros((batch_size, cutoff, 1, 128, 9))
        batch_yields = np.zeros(batch_size)

        if batches == len(IDs) // batch_size:
            batches = 0
            yield None, None

        for i in range(batch_size):
            index = random.choice(range(len(IDs)))
            ID = IDs[index]
            data = load_data(ID)

            if data is not None:
                if cutoff is not None:
                    if not np.isnan(data).any():
                        batch_features[i, :, :, :, :] = data[:cutoff, :, :, :]
                        batch_yields[i] = yields[ID]
                    else:
                        print('Data contains NaN values:', ID)
                else:
                    batch_features[i, :, :, :, :] = data
                    batch_yields[i] = yields[ID]
                

        batches += 1

        yield torch.tensor(batch_features, dtype=torch.float32, device='cuda'), torch.tensor(batch_yields, dtype=torch.float32, device='cuda')


In [4]:
# Datasets
yields = pickle.load(open('data/yields.p', 'rb'))

# Generators
training_generator = generator(list(yields['train'].keys()), yields['train'], 16)
validation_generator = generator(list(yields['validation'].keys()), yields['validation'], 16)


In [6]:
n_estimators_values = [50, 100, 150]  
max_depth_values = [None, 10, 20]

# Initialize variables to store best parameters and corresponding MSE
best_params = None
best_mse = float('inf')  

for n_estimators in n_estimators_values:
    for max_depth in max_depth_values:
        random_forest_model = RandomForestBaseline(n_estimators=n_estimators, max_depth=max_depth, random_state=42)

        # Fit the model to training data
        X_train, y_train = next(training_generator)
        random_forest_model.fit(X_train.cpu().reshape(X_train.shape[0], -1), y_train.cpu())

        # Make predictions on test data
        X_test, y_test = next(validation_generator)
        predictions = random_forest_model.predict(X_test.cpu().reshape(X_test.shape[0], -1))

        # Evaluate the model
        mse = random_forest_model.evaluate(X_test.cpu().reshape(X_test.cpu().shape[0], -1), y_test.cpu())

        # Print MSE for current parameter combination
        print(f"Parameters: n_estimators={n_estimators}, max_depth={max_depth} MSE: {mse}")

        # Check if current combination improves performance
        if mse < best_mse:
            print('save best model')
            joblib.dump(random_forest_model, 'random_forest_best_model.pkl')            
            best_mse = mse
            best_params = {'n_estimators': n_estimators, 'max_depth': max_depth}

# Print best parameters and corresponding MSE
print("\nBest Parameters:")
print(best_params)
print("Best Mean Squared Error:", best_mse)

Parameters: n_estimators=50, max_depth=None MSE: 619.354345526888
save best model
Parameters: n_estimators=50, max_depth=10 MSE: 1808.0978195844714
Parameters: n_estimators=50, max_depth=20 MSE: 2259.248291233861
Parameters: n_estimators=100, max_depth=None MSE: 1168.7079847656044
Parameters: n_estimators=100, max_depth=10 MSE: 1409.55830251155
Parameters: n_estimators=100, max_depth=20 MSE: 1634.5533773545549
Parameters: n_estimators=150, max_depth=None MSE: 275.9240671405978
save best model
Parameters: n_estimators=150, max_depth=10 MSE: 4696.29946511155
Parameters: n_estimators=150, max_depth=20 MSE: 1153.1625805266085

Best Parameters:
{'n_estimators': 150, 'max_depth': None}
Best Mean Squared Error: 275.9240671405978


In [ ]:
alpha_values = [0.3, 0.4, 0.5]

## for Lasso baseline model
lasso_model = LassoModel(alpha=0.5, random_state=42)

# Fit the model to training data
X_train, y_train = next(training_generator)
lasso_model.fit(X_train.cpu().reshape(X_train.shape[0], -1), y_train.cpu())

# Make predictions on test data
X_test, y_test = next(validation_generator)
predictions = lasso_model.predict(X_test.cpu().reshape(X_test.shape[0], -1))  # Flattening input features

# Evaluate the model
mse = lasso_model.evaluate(X_test.cpu().reshape(X_test.cpu().shape[0], -1), y_test.cpu())  # Flattening input features
print("Mean Squared Error:", mse)

Mean Squared Error: 3469.941841439345


In [ ]:
alpha_values = [0.3, 0.4, 0.5]

best_alpha = None
best_mse = float('inf') 

# Iterate over alpha values
for alpha in alpha_values:
    # Initialize Lasso model with current alpha value
    lasso_model = LassoModel(alpha=alpha, random_state=42)

    # Fit the model to training data
    X_train, y_train = next(training_generator)
    lasso_model.fit(X_train.cpu().reshape(X_train.shape[0], -1), y_train.cpu())

    # Make predictions on test data
    X_test, y_test = next(validation_generator)
    predictions = lasso_model.predict(X_test.cpu().reshape(X_test.shape[0], -1))

    # Evaluate the model
    mse = lasso_model.evaluate(X_test.cpu().reshape(X_test.cpu().shape[0], -1), y_test.cpu())

    print(f"Alpha: {alpha}, MSE: {mse}")

    # Check if current alpha improves performance
    if mse < best_mse:
        print('save best model')
        joblib.dump(lasso_model, 'lasso_best_model.pkl')   
        best_mse = mse
        best_alpha = alpha

# Print best alpha and corresponding MSE
print("\nBest Alpha:", best_alpha)
print("Best Mean Squared Error:", best_mse)


In [19]:
model_functions = {
    'CNN_LSTM': CNN_LSTM,
    'SepCNN_LSTM': SepCNN_LSTM,
    'ConvGRU_LSTM': ConvGRU_LSTM,
}


epochs = 50

for model_name, model_function in model_functions.items():
    model = model_function(dimensions=[38, 1, 128, 9])
    model.to('cuda')
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.MSELoss()

    best_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        train_losses = []
        for batch_data, batch_labels in tqdm.tqdm(training_generator, desc=f"Epoch {epoch+1}/{epochs}"):
            if batch_data is None:
                break

            optimizer.zero_grad()
            outputs = model(batch_data)
            loss = criterion(outputs, batch_labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()
        val_losses = []
        
        with torch.no_grad():
            for val_data, val_labels in validation_generator:
                if val_data is None:
                    break
                val_outputs = model(val_data)
                val_loss = criterion(val_outputs, val_labels.unsqueeze(1))
                val_losses.append(val_loss.item())

        current_loss = np.mean(val_losses)
        if current_loss < best_loss:
            print('save best model')
            torch.save(model, f'{model_name}_best.pt')
            best_loss = current_loss
            
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {np.mean(train_losses):.4f}, Val Loss: {current_loss:.4f}")

Epoch 1/50: 95it [00:02, 34.49it/s]


save best model
Epoch 1/50, Train Loss: 4979.9436, Val Loss: 7690.8474


Epoch 2/50: 389it [00:11, 35.04it/s]


save best model
Epoch 2/50, Train Loss: 2964.7165, Val Loss: 6679.4941


Epoch 3/50: 389it [00:11, 34.70it/s]


...

Epoch 48/50, Train Loss: 508.6565, Val Loss: 428.4117


Epoch 49/50: 389it [00:27, 14.29it/s]


Epoch 49/50, Train Loss: 539.7897, Val Loss: 541.8752


Epoch 50/50: 389it [00:27, 14.33it/s]


Epoch 50/50, Train Loss: 546.1499, Val Loss: 1126.5159
